In [1]:
import pandas as pd
# from speechbrain.inference.speaker import SpeakerRecognition
import os
from pydub import AudioSegment
import speechbrain
import torch
import torchaudio
import random

from speechbrain.augment.time_domain import Resample
from speechbrain.dataio.dataio import read_audio
import torchaudio.transforms as T
from speechbrain.lobes.features import Fbank
from speechbrain.processing.features import InputNormalization
from pytorch_metric_learning import losses


/home/ivbeveren/.local/lib/python3.11/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
torchvision is not available - cannot save figures


In [56]:
!pip install pytorch-metric-learning

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.1/119.1 kB 24.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
from speechbrain.dataio.dataset import DynamicItemDataset
dataset_train = DynamicItemDataset.from_csv("UCLA-train.csv") # or equivalently, DynamicItemDataset.from_csv("data.csv")
# dataset_val = DynamicItemDataset.from_csv("UCLA-val.csv") # or equivalently, DynamicItemDataset.from_csv("data.csv")

In [3]:
@speechbrain.utils.data_pipeline.takes("path", "rescaled_speakerID")
@speechbrain.utils.data_pipeline.provides("signal1", "lab")
def audio_pipeline(path, rescaled_speakerID, max_duration=10):
    waveform1, sample_rate1 = torchaudio.load(path.replace("\\", "/"))

    # feature_maker = Fbank(n_mels = 80, left_frames = 0, right_frames = 0, deltas = False)
    # norm = InputNormalization(norm_type = 'sentence', std_norm = False, )

    # Resample the audio to 16 kHz if needed
    resampler = speechbrain.augment.time_domain.Resample(orig_freq=sample_rate1, new_freq=16000)
    waveform1 = resampler(waveform1)
    max_samples = int(max_duration * 16000)  # Maximum number of samples allowed
    if waveform1.size(1) > max_samples:
        waveform1 = waveform1[:, :max_samples]
    
    waveform1 = waveform1.transpose(0, 1).squeeze(1)

    
    # # Limit the duration if needed

    
    # features1 = feature_maker(waveform1)
    
    # inp_len1 = torch.Tensor([features1.shape[1]])
    # features1 = norm(features1, inp_len1)



    return waveform1, int(rescaled_speakerID)


In [109]:
# torchaudio.load('UCLA\\001\\1A_instructions.wav')
os.path.isfile('UCLA/001/1A_instructions.wav')

True

In [4]:
dataset_train.add_dynamic_item(audio_pipeline)
# dataset_val.add_dynamic_item(audio_pipeline)

dataset_train.set_output_keys(["signal1", "lab"])
# dataset_val.set_output_keys(["signal1", "lab"])

In [111]:
len(dataset_train)

1098

In [118]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
# labels = [item['lab'] for item in dataset]
# print(labels)
# Perform train, validation, and test split
# labels = [data['lab'] for data in dataset]
train_data, val_data = train_test_split(dataset_train, test_size=0.2, random_state=42)
# val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

In [119]:
from collections import defaultdict
from speechbrain.dataio.dataset import DynamicItemDataset
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

from collections import defaultdict

# Group indices by label
label_to_indices_train = defaultdict(list)
for idx, data in enumerate(train_data):
    label = data['lab']
    label_to_indices_train[label].append(idx)

label_to_indices_val = defaultdict(list)
for idx, data in enumerate(val_data):
    label = data['lab']
    label_to_indices_val[label].append(idx)
import numpy as np

def create_batches(label_to_indices, batch_size):
    batches = []
    for label, indices in label_to_indices.items():
        np.random.shuffle(indices)  # Shuffle to ensure randomness
        for i in range(0, len(indices), 2):
            if i + 1 < len(indices):
                batch = indices[i:i+2]
                # Fill the rest of the batch with random pairs
                while len(batch) < batch_size:
                    rand_label = np.random.choice(list(label_to_indices.keys()))
                    if len(label_to_indices[rand_label]) >= 2:
                        rand_indices = np.random.choice(label_to_indices[rand_label], 2, replace=False).tolist()
                    else:
                        rand_indices = np.random.choice(label_to_indices[rand_label], 2, replace=True).tolist()
                    batch.extend(rand_indices[:batch_size - len(batch)])  # Ensure batch size is not exceeded
                batches.append(batch)
    return batches
batches_train = create_batches(label_to_indices_train, batch_size=16)
batches_val = create_batches(label_to_indices_val, batch_size=16)
from torch.utils.data import Sampler

class CustomBatchSampler(Sampler):
    def __init__(self, batches):
        self.batches = batches
    
    def __iter__(self):
        np.random.shuffle(self.batches)
        for batch in self.batches:
            yield batch
    
    def __len__(self):
        return len(self.batches)

custom_batch_sampler_train = CustomBatchSampler(batches_train)
custom_batch_sampler_val = CustomBatchSampler(batches_val)

In [3]:
from speechbrain.dataio.dataloader import SaveableDataLoader
from speechbrain.dataio.batch import PaddedBatch

dataloader_train = DataLoader(train_data, batch_sampler=custom_batch_sampler_train, collate_fn=PaddedBatch)
dataloader_val = DataLoader(val_data, batch_sampler=custom_batch_sampler_val, collate_fn=PaddedBatch)

NameError: name 'DataLoader' is not defined

In [123]:
for batch in dataloader_train:
    print(batch.lab)


tensor([163, 163,  59,  59,  91,  91, 111, 111, 121, 121,  75,  75,   4,   4,
         14,  14])
tensor([ 26,  26,   1,   1,  80,  80,  83,  83, 164, 164,  31,  31,  45,  45,
        125, 125])
tensor([112, 112,  26,  26, 162, 162, 141, 141, 123, 123,  31,  31, 172, 172,
         54,  54])
tensor([ 76,  76,  14,  14, 139, 139, 102, 102,  18,  18, 131, 131,  53,  53,
         88,  88])
tensor([ 77,  77, 171, 171,  95,  95, 143, 143, 144, 144,   0,   0,  83,  83,
        148, 148])
tensor([ 46,  46, 133, 133,  62,  62,   6,   6, 112, 112, 170, 170, 165, 165,
         31,  31])
tensor([109, 109,  37,  37,  35,  35,  13,  13, 124, 124,  96,  96, 150, 150,
         37,  37])
tensor([113, 113, 114, 114, 104, 104, 125, 125,  31,  31, 105, 105, 119, 119,
         50,  50])
tensor([165, 165,  41,  41, 110, 110,  37,  37,  13,  13, 161, 161,  94,  94,
         68,  68])
tensor([ 28,  28, 171, 171, 136, 136,   2,   2,  53,  53,  53,  53,  25,  25,
        114, 114])
tensor([ 52,  52, 147, 147,  3

In [9]:
from collections import Counter

# Assuming your labels are stored in a list called 'labels'
labels = [item['lab'] for item in dataset_train]

# Count the occurrences of each label
label_distribution = Counter(labels)

# Print the label distribution
print(label_distribution)

Counter({99: 9, 28: 8, 52: 8, 91: 8, 92: 8, 122: 8, 123: 8, 129: 8, 169: 8, 2: 7, 4: 7, 6: 7, 8: 7, 19: 7, 20: 7, 30: 7, 33: 7, 36: 7, 37: 7, 39: 7, 42: 7, 46: 7, 47: 7, 48: 7, 56: 7, 61: 7, 62: 7, 65: 7, 67: 7, 74: 7, 76: 7, 77: 7, 79: 7, 85: 7, 86: 7, 87: 7, 88: 7, 90: 7, 93: 7, 95: 7, 96: 7, 104: 7, 105: 7, 107: 7, 109: 7, 113: 7, 114: 7, 115: 7, 119: 7, 120: 7, 121: 7, 126: 7, 130: 7, 133: 7, 134: 7, 136: 7, 137: 7, 138: 7, 148: 7, 151: 7, 155: 7, 156: 7, 159: 7, 162: 7, 164: 7, 166: 7, 167: 7, 173: 7, 0: 6, 1: 6, 3: 6, 5: 6, 7: 6, 9: 6, 11: 6, 13: 6, 14: 6, 16: 6, 18: 6, 22: 6, 23: 6, 25: 6, 27: 6, 31: 6, 32: 6, 34: 6, 35: 6, 38: 6, 40: 6, 43: 6, 44: 6, 45: 6, 50: 6, 51: 6, 53: 6, 55: 6, 58: 6, 59: 6, 60: 6, 63: 6, 64: 6, 66: 6, 68: 6, 70: 6, 71: 6, 72: 6, 73: 6, 75: 6, 78: 6, 80: 6, 81: 6, 82: 6, 83: 6, 84: 6, 94: 6, 102: 6, 103: 6, 106: 6, 108: 6, 110: 6, 111: 6, 116: 6, 117: 6, 124: 6, 125: 6, 127: 6, 128: 6, 131: 6, 132: 6, 135: 6, 140: 6, 141: 6, 142: 6, 143: 6, 144: 6, 145: 

In [10]:
train_labels = [item['lab'] for item in train_data]

# Extract labels from val_data
val_labels = [item['lab'] for item in val_data]

# Get unique labels in train and validation sets
unique_train_labels = set(train_labels)
unique_val_labels = set(val_labels)

print("Unique labels in the training set:", len(unique_train_labels))
print("Unique labels in the validation set:", unique_val_labels)


Unique labels in the training set: 174
Unique labels in the validation set: {0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 22, 23, 26, 28, 29, 30, 31, 32, 33, 34, 36, 37, 38, 39, 40, 42, 43, 44, 45, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 61, 64, 65, 66, 67, 68, 70, 71, 72, 73, 74, 75, 77, 78, 79, 80, 81, 83, 84, 85, 86, 87, 88, 91, 93, 95, 96, 99, 101, 103, 105, 106, 107, 108, 111, 112, 113, 114, 115, 117, 118, 119, 120, 121, 122, 123, 124, 126, 127, 129, 130, 131, 133, 134, 135, 136, 137, 138, 143, 144, 145, 146, 148, 149, 150, 151, 152, 153, 155, 158, 160, 161, 162, 163, 164, 166, 167, 168, 169, 170, 172, 173}


In [65]:
from speechbrain.inference.speaker import SpeakerRecognition
import torchaudio
verification = SpeakerRecognition.from_hparams(source="speechbrain/spkrec-resnet-voxceleb", savedir="pretrained_models/spkrec-resnet-voxceleb")


In [124]:
from speechbrain.lobes.models.ResNet import ResNet
from speechbrain.utils.parameter_transfer import Pretrainer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ResNet(input_size=80, 
               device=device, 
               channels=[128, 128, 256, 256], 
               block_sizes=[3, 4, 6, 3], 
               strides=[1, 2, 2, 2], 
               lin_neurons=256)

pretrainer = speechbrain.utils.parameter_transfer.Pretrainer(loadables={'model': model}, paths={'model': 'speechbrain/spkrec-resnet-voxceleb/embedding_model.ckpt'})
pretrainer.collect_files
pretrainer.load_collected()

model.to(device)

ResNet(
  (conv1): Conv2d(1, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activation1): ReLU()
  (layer1): Sequential(
    (0): SEBasicBlock(
      (activation): ReLU()
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (se): SEBlock(
        (avg_pool): AdaptiveAvgPool2d(output_size=1)
        (fc): Sequential(
          (0): Linear(in_features=128, out_features=128, bias=Tr

In [33]:
for param in model.parameters():
    param.requires_grad = False


In [89]:
from torch import nn
class AdMSoftmaxLoss(torch.nn.Module):

    def __init__(self, in_features, out_features, s=30.0, m=0.4):
        '''
        AM Softmax Loss
        '''
        super(AdMSoftmaxLoss, self).__init__()
        self.s = s
        self.m = m
        self.in_features = in_features
        self.out_features = out_features
        self.fc = nn.Linear(in_features, out_features, bias=False)

    def forward(self, x, labels):
        '''
        input shape (N, in_features)
        '''
        assert len(x) == len(labels)
        assert torch.min(labels) >= 0
        assert torch.max(labels) < self.out_features
        
        for W in self.fc.parameters():
            W = F.normalize(W, dim=1)

        x = F.normalize(x, dim=1)

        wf = self.fc(x)
        numerator = self.s * (torch.diagonal(wf.transpose(0, 1)[labels]) - self.m)
        excl = torch.cat([torch.cat((wf[i, :y], wf[i, y+1:])).unsqueeze(0) for i, y in enumerate(labels)], dim=0)
        denominator = torch.exp(numerator) + torch.sum(torch.exp(self.s * excl), dim=1)
        L = numerator - torch.log(denominator)
        return -torch.mean(L)

In [10]:
from speechbrain.nnet.losses import AdditiveAngularMargin
from speechbrain.utils.metric_stats import EER
from speechbrain.utils.distributed import run_on_main
from speechbrain.utils.checkpoints import Checkpointer
from speechbrain.utils import epoch_loop
import torch
import torch.nn.functional as F

in_features = 256  # This should match the output size of the last layer of the model
out_features = 2   # Number of classes (binary classification)
criterion = AdMSoftmaxLoss(in_features, out_features, s=30.0, m=0.4).to(device)

# Define the loss function and optimizer
# loss_fn = AdditiveAngularMargin()

# pretrained_params = list(model.layer2.parameters()) + list(model.layer3.parameters()) + list(model.layer4.parameters())
# new_params = list(model.fc_embed.parameters())

# param_groups = [
#     {'params': pretrained_params, 'lr': 0.005},
#     {'params': new_params, 'lr': 0.001}
# ]

# Initialize the optimizer with parameter groups
optimizer = torch.optim.Adam(criterion.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30, eta_min=0.001)


# Initialize the checkpointer
checkpointer = Checkpointer(checkpoints_dir="checkpoint_dir", recoverables={'model': model, 'optimizer': optimizer})


NameError: name 'AdMSoftmaxLoss' is not defined

In [12]:
import torch
import torch.nn.functional as F

def nt_bxent_loss(x, pos_indices, temperature):
    assert len(x.size()) == 2

    # Add indexes of the principal diagonal elements to pos_indices
    pos_indices = torch.cat([
        pos_indices,
        torch.arange(x.size(0)).reshape(x.size(0), 1).expand(-1, 2).to(x.device),
    ], dim=0)

    # Ground truth labels
    target = torch.zeros(x.size(0), x.size(0), device=x.device)
    target[pos_indices[:,0], pos_indices[:,1]] = 1.0

    # Cosine similarity
    xcs = F.cosine_similarity(x[None,:,:], x[:,None,:], dim=-1)
    # Set logit of diagonal element to "inf" signifying complete
    # correlation. sigmoid(inf) = 1.0 so this will work out nicely
    # when computing the Binary Cross Entropy Loss.
    xcs[torch.eye(x.size(0), device=x.device).bool()] = float("inf")

    # Standard binary cross entropy loss
    loss = F.binary_cross_entropy((xcs / temperature).sigmoid(), target, reduction="none")

    target_pos = target.bool()
    target_neg = ~target_pos

    loss_pos = torch.zeros(x.size(0), x.size(0), device=x.device).masked_scatter(target_pos, loss[target_pos])
    loss_neg = torch.zeros(x.size(0), x.size(0), device=x.device).masked_scatter(target_neg, loss[target_neg])
    loss_pos = loss_pos.sum(dim=1)
    loss_neg = loss_neg.sum(dim=1)
    num_pos = target.sum(dim=1)
    num_neg = x.size(0) - num_pos

    return ((loss_pos / num_pos) + (loss_neg / num_neg)).mean()


In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NTXentLoss(nn.Module):
    def __init__(self, temperature=0.5):
        super(NTXentLoss, self).__init__()
        self.temperature = temperature

    def forward(self, z_i, z_j):
        batch_size = z_i.size(0)

        # Ensure the inputs are float tensors
        z_i = z_i.float()
        z_j = z_j.float()

        z_i = F.normalize(z_i, dim=1)
        z_j = F.normalize(z_j, dim=1)

        representations = torch.cat([z_i, z_j], dim=0)
        similarity_matrix = F.cosine_similarity(representations.unsqueeze(1), representations.unsqueeze(0), dim=2)

        sim_ij = torch.diag(similarity_matrix, batch_size)
        sim_ji = torch.diag(similarity_matrix, -batch_size)

        positive_samples = torch.cat([sim_ij, sim_ji], dim=0)
        positive_samples = positive_samples / self.temperature

        negative_samples = similarity_matrix / self.temperature

        labels = torch.arange(2 * batch_size).to(z_i.device)
        loss = F.cross_entropy(negative_samples, labels)

        return loss.mean()



## softmax layer training ##

In [49]:
num_epochs = 30
for epoch in range(num_epochs):
    model.eval()  # Pre-trained layers are frozen, so no need for dropout or batchnorm in training mode
    # criterion.train()
    for batch in dataloader_train:
        signal1, signal2, labels = batch["signal1"], batch["signal2"], batch["lab"]
        
        signal1 = signal1.data.to(device)
        signal2 = signal2.data.to(device)
        labels = torch.tensor(labels, dtype=torch.long).to(device)

        with torch.no_grad():
            embeddings1 = model(signal1)
            embeddings2 = model(signal2)
                        
        # Create the one-hot encoded targets
#         targets = F.one_hot(labels, num_classes=2).float().to(device)
        
        
        embeddings = torch.cat((embeddings1, embeddings2), dim=0)
        # AM softmax loss
        labels = torch.cat((labels, labels), dim=0)
        loss = criterion(embeddings, labels)
        
        # AAM softmax loss
        cosine_sim = F.cosine_similarity(embeddings1, embeddings2)

        # Apply the AAM loss function
        loss = loss_fn(cosine_sim, targets)      
        loss = abs(loss.mean())

          # NT-Xent loss
#         batch_size = signal1.size(0)
#         pos_indices = torch.arange(batch_size).unsqueeze(1).repeat(1, 2)
        
#         Apply the NT-Xent loss function
#         loss = nt_bxent_loss(embeddings.to(device), pos_indices.to(device), temperature=0.5)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
#         print("did a train loop")

    scheduler.step()
    checkpointer.save_and_keep_only(meta={"loss": loss.item()}, num_to_keep=1)
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

# Save the final model
torch.save(model.state_dict(), "fine_tuned_model.pth")

/scratch-local/ivbeveren.6569262/ipykernel_880169/3766782426.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.long).to(device)


Epoch 1/30, Loss: 9.323351860046387
Epoch 2/30, Loss: 8.885387420654297
Epoch 3/30, Loss: 8.829681396484375
Epoch 4/30, Loss: 8.967159271240234
Epoch 5/30, Loss: 9.028463363647461
Epoch 6/30, Loss: 9.044422149658203
Epoch 7/30, Loss: 9.056853294372559
Epoch 8/30, Loss: 9.045794486999512
Epoch 9/30, Loss: 9.01041030883789
Epoch 10/30, Loss: 8.958152770996094
Epoch 11/30, Loss: 8.894526481628418
Epoch 12/30, Loss: 8.825304985046387
Epoch 13/30, Loss: 8.755851745605469
Epoch 14/30, Loss: 8.690963745117188
Epoch 15/30, Loss: 8.634889602661133
Epoch 16/30, Loss: 8.590059280395508
Epoch 17/30, Loss: 8.557659149169922
Epoch 18/30, Loss: 8.538214683532715
Epoch 19/30, Loss: 8.531195640563965
Epoch 20/30, Loss: 8.535123825073242
Epoch 21/30, Loss: 8.548553466796875
Epoch 22/30, Loss: 8.571151733398438
Epoch 23/30, Loss: 8.603395462036133
Epoch 24/30, Loss: 8.644685745239258
Epoch 25/30, Loss: 8.691736221313477
Epoch 26/30, Loss: 8.738676071166992
Epoch 27/30, Loss: 8.778890609741211
Epoch 28/30

## Full nn training ##

In [125]:
# for param in model.parameters():
#     param.requires_grad = True
from speechbrain.nnet.losses import AdditiveAngularMargin
from speechbrain.utils.metric_stats import EER
from speechbrain.utils.distributed import run_on_main
from speechbrain.utils.checkpoints import Checkpointer
from speechbrain.utils import epoch_loop
import torch
import torch.nn.functional as F
from speechbrain.nnet.losses import LogSoftmaxWrapper, AdditiveAngularMargin
from speechbrain.lobes.models.ResNet import Classifier


# for param in model.conv1.parameters():
#     param.requires_grad = False
# for param in model.layer1.parameters():
#     param.requires_grad = False
# Define parameter groups with different learning rates
# model.fc_embed = torch.nn.Linear(in_features=5120, out_features=256, bias=True, device=device)
# for param in model.fc_embed.parameters():
#     param.requires_grad = True

pretrained_params = list(model.layer1.parameters()) + list(model.layer2.parameters()) + list(model.layer3.parameters()) + list(model.layer4.parameters())
new_params = list(model.fc_embed.parameters())
criterion = losses.NTXentLoss(temperature=0.07)
param_groups = [
    {'params': pretrained_params, 'lr': 0.005},
    {'params': new_params, 'lr': 0.001},
    # {'params': list(criterion.parameters()), 'lr':0.001}   
]

# Re-initialize the optimizer with parameter groups
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30, eta_min=0.001)
checkpointer = Checkpointer(checkpoints_dir="checkpoint_dir", recoverables={'model': model, 'optimizer': optimizer})
clas = Classifier(input_size = 256, out_neurons = 174).to(device)

In [72]:
for batch in dataloader_train:
    signal1, labels = batch["signal1"], batch["lab"]
    labels = torch.LongTensor(labels).unsqueeze(1)
    signal1 = signal1.data
    # Convert labels to tensor and move to GPU
    optimizer.zero_grad()
    embeddings1 = model(signal1)
    output = clas(embeddings1)
    print(output.shape)
    print(labels.shape)
    loss = criterion(output, labels)
    print(loss)
    break

torch.Size([8, 1, 174])
torch.Size([8, 1])
tensor(13.0202, grad_fn=<DivBackward0>)


In [126]:
early_stopping_patience = 8
best_val_loss = float('inf')
patience_counter = 0
num_epochs = 30

feature_maker = Fbank(n_mels = 80, left_frames=0, right_frames=0, deltas=False)
norm = InputNormalization(norm_type = 'sentence', std_norm = False)
cosine_sim = torch.nn.CosineSimilarity(dim=-1, eps=1e-6)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch in dataloader_train:
        batch = batch.to(device)
        signal1, lens1 = batch.signal1
        labels = batch.lab
        # Convert labels to tensor and move to GPU
        # labels = torch.tensor(labels, dtype=torch.long).to(device)
        # labels = labels.unsqueeze(1)
        if len(signal1.shape) == 1:
            signal1 = signal1.unsqueeze(0)
    
        if lens1 is None:
            lens1 = torch.ones(signal1.shape[0], device=device)
    
        signal1, lens1 = signal1.to(device), lens1.to(device)
        signal1 = signal1.float()
        
        features1 = feature_maker(signal1)    
        features1 = norm(features1, lens1)
        
        optimizer.zero_grad()
        embeddings1 = model(features1)

        # output = clas(embeddings1)
        # Apply the AAM loss function
        loss = criterion(embeddings1, labels)
                
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
    epoch_loss = running_loss / len(dataloader_train)
    # scheduler.step()
    checkpointer.save_and_keep_only(meta={"loss": loss.item()}, num_to_keep=1)
    print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {epoch_loss}')

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in dataloader_val:
            batch = batch.to(device)
            signal1, lens1 = batch.signal1
            labels = batch.lab
            # Convert labels to tensor and move to GPU
            # labels = torch.tensor(labels, dtype=torch.long).to(device)
            # labels = labels.unsqueeze(1)
            if len(signal1.shape) == 1:
                signal1 = signal1.unsqueeze(0)

            
        
            if lens1 is None:
                lens1 = torch.ones(signal1.shape[0], device=device)
        
            signal1, lens1 = signal1.to(device), lens1.to(device)
            signal1 = signal1.float()
            
            features1 = feature_maker(signal1)    
            features1 = norm(features1, lens1)
            
            embeddings1 = model(features1)
    
            # output = clas(embeddings1)
            # Apply the AAM loss function
            loss = criterion(embeddings1, labels)
            val_loss += loss.item()
    
    val_loss /= len(dataloader_val)
    print(f'Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss}')

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(model.state_dict(), "fine_tuned_model.pth")  # Save the best model
        print("Model saved!")
    else:
        patience_counter += 1
        if patience_counter >= early_stopping_patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

torch.save(model.state_dict(), "fine_tuned_model.pth")

Epoch 1/30, Training Loss: 0.052221337602550426
Epoch 1/30, Validation Loss: 0.09677804583412554
Model saved!
Epoch 2/30, Training Loss: 0.0016915484258236033
Epoch 2/30, Validation Loss: 0.08024667577779811
Model saved!
Epoch 3/30, Training Loss: 0.0008150713358375043
Epoch 3/30, Validation Loss: 0.08101093056420587
Epoch 4/30, Training Loss: 0.0005562579951137068
Epoch 4/30, Validation Loss: 0.08202412839172714
Epoch 5/30, Training Loss: 0.0004105561586388658
Epoch 5/30, Validation Loss: 0.08210566590554011
Epoch 6/30, Training Loss: 0.0003164770810708663
Epoch 6/30, Validation Loss: 0.08042430546423142
Epoch 7/30, Training Loss: 0.00025089047187895077
Epoch 7/30, Validation Loss: 0.08309138129350604
Epoch 8/30, Training Loss: 0.00020175675885115245
Epoch 8/30, Validation Loss: 0.08014193997299646
Model saved!


KeyboardInterrupt: 

In [128]:
torch.save(model.state_dict(), "fine_tuned_model.pth")

In [88]:
# Function to interpolate two models
def interpolate_models(model1, model2, alpha):
    # Ensure the models are on the same device
    device = next(model1.parameters()).device
    model2.to(device)
    
    # Create a new model with the same architecture
    interpolated_model = type(model1)().to(device)
    
    # Interpolate the parameters
    for param1, param2, param_interpolated in zip(model1.parameters(), model2.parameters(), interpolated_model.parameters()):
        param_interpolated.data = alpha * param1.data + (1 - alpha) * param2.data
    
    # Interpolate the buffers (including BatchNorm running mean/var)
    for buffer1, buffer2, buffer_interpolated in zip(model1.buffers(), model2.buffers(), interpolated_model.buffers()):
        buffer_interpolated.data = alpha * buffer1.data + (1 - alpha) * buffer2.data
    
    return interpolated_model

In [89]:
from speechbrain.lobes.models.ResNet import ResNet
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
pretrained_model = ResNet(input_size=80, 
               device=device, 
               channels=[128, 128, 256, 256], 
               block_sizes=[3, 4, 6, 3], 
               strides=[1, 2, 2, 2], 
               lin_neurons=256)



cuda


In [90]:
from speechbrain.utils.parameter_transfer import Pretrainer
pretrain = Pretrainer(loadables={'model': pretrained_model}, paths={'model': 'speechbrain/spkrec-resnet-voxceleb/embedding_model.ckpt'})

# We download the pretrained model from HuggingFace in this case
pretrain.collect_files()
pretrain.load_collected()


pretrained_model.to(device)

ResNet(
  (conv1): Conv2d(1, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activation1): ReLU()
  (layer1): Sequential(
    (0): SEBasicBlock(
      (activation): ReLU()
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (se): SEBlock(
        (avg_pool): AdaptiveAvgPool2d(output_size=1)
        (fc): Sequential(
          (0): Linear(in_features=128, out_features=128, bias=Tr

In [91]:
finetuned_model = ResNet(input_size=80, 
               device=device, 
               channels=[128, 128, 256, 256], 
               block_sizes=[3, 4, 6, 3], 
               strides=[1, 2, 2, 2], 
               lin_neurons=256)
finetuned_model.load_state_dict(torch.load("fine_tuned_model.pth"))
finetuned_model.to(device)

ResNet(
  (conv1): Conv2d(1, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activation1): ReLU()
  (layer1): Sequential(
    (0): SEBasicBlock(
      (activation): ReLU()
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (se): SEBlock(
        (avg_pool): AdaptiveAvgPool2d(output_size=1)
        (fc): Sequential(
          (0): Linear(in_features=128, out_features=128, bias=Tr

In [92]:
alpha = 0.3  # Change alpha as needed
interpolated_model = interpolate_models(pretrained_model, finetuned_model, alpha)

# Move the interpolated model to the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
interpolated_model.to(device)

ResNet(
  (conv1): Conv2d(1, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activation1): ReLU()
  (layer1): Sequential(
    (0): SEBasicBlock(
      (activation): ReLU()
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (se): SEBlock(
        (avg_pool): AdaptiveAvgPool2d(output_size=1)
        (fc): Sequential(
          (0): Linear(in_features=128, out_features=128, bias=Tr

In [93]:
for param in interpolated_model.parameters():
    print(param)
    break

for param in finetuned_model.parameters():
    print(param)
    break
for param in pretrained_model.parameters():
    print(param)
    break

Parameter containing:
tensor([[[[ 0.0905,  0.2573,  0.3710],
          [ 0.1229,  0.7042,  1.0147],
          [ 0.1084, -0.1089,  0.6465]]],


        [[[-0.0724,  0.1205,  0.1229],
          [-0.0572, -0.0666,  0.0087],
          [-0.3296, -0.4549, -0.3854]]],


        [[[-0.1603,  0.0503, -0.2438],
          [-0.2613, -0.1426, -0.2535],
          [-0.1318, -0.2272,  0.1841]]],


        ...,


        [[[-0.0702, -0.0602, -0.3591],
          [ 0.0137, -0.1686,  1.3274],
          [ 0.0493,  0.0901, -0.6045]]],


        [[[ 0.1862, -0.8853,  0.3028],
          [ 0.0665, -0.4076,  0.1459],
          [-0.3951,  0.2630, -0.0063]]],


        [[[-0.3060, -0.0299,  0.3859],
          [-0.7436, -0.2048,  0.7486],
          [-0.5148, -0.0582,  0.6231]]]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([[[[ 0.0893,  0.2528,  0.3696],
          [ 0.1233,  0.7026,  1.0174],
          [ 0.1099, -0.1096,  0.6480]]],


        [[[-0.0793,  0.1139,  0.1170],
          [-0.0640,

In [71]:
torch.save(interpolated_model.state_dict(), "interpolated_model.pth")

In [118]:
@speechbrain.utils.data_pipeline.takes("file1", "file2", "label")
@speechbrain.utils.data_pipeline.provides("signal1", "signal2", "lab")
def audio_pipeline_verification(file1, file2, label, max_duration=1000):
    waveform1, sample_rate1 = torchaudio.load(file1.replace("\\", "/"))
    waveform2, sample_rate2 = torchaudio.load(file2.replace("\\", "/"))
    # feature_maker = Fbank(n_mels = 80)
    # norm = InputNormalization(norm_type = 'sentence', std_norm = False)

    # Resample the audio to 16 kHz if needed
    # resampler = speechbrain.augment.time_domain.Resample(orig_freq=sample_rate1, new_freq=16000)
    # waveform1 = resampler(waveform1)
    # waveform2 = resampler(waveform2)
    
    # Limit the duration if needed
    max_samples = int(max_duration * 16000)  # Maximum number of samples allowed
    if waveform1.size(1) > max_samples:
        waveform1 = waveform1[:, :max_samples]
    if waveform2.size(1) > max_samples:
        waveform2 = waveform2[:, :max_samples]

    waveform1 = waveform1.transpose(0, 1).squeeze(1)
    waveform2 = waveform2.transpose(0, 1).squeeze(1)

    # Convert labels from 'true'/'false' to 1/0
    # label = 1 if label == 'True' else 0
   
    
    # features1 = feature_maker(waveform1)
    # features2 = feature_maker(waveform2)
    
    # inp_len1 = torch.Tensor([features1.shape[1]])
    # inp_len2 = torch.Tensor([features2.shape[1]])
    # features1 = norm(features1, features1.lens)
    # features2 = norm(features2, features2.lens)

    return waveform1, waveform2, label

In [119]:
dataset_ver = DynamicItemDataset.from_csv("Wyred-pairs-nolarge-cut.csv") # or equivalently, DynamicItemDataset.from_csv("data.csv")

In [120]:
dataset_ver.add_dynamic_item(audio_pipeline_verification)
dataset_ver.set_output_keys(["signal1", "signal2", "lab"])
dataloader_test = SaveableDataLoader(dataset_ver, batch_size=2, collate_fn=PaddedBatch)

In [121]:
dataset_ver[0]

{'signal1': tensor([-0.0009, -0.0009,  0.0000,  ...,  0.0009,  0.0046,  0.0083]),
 'signal2': tensor([ 0.0102,  0.0103,  0.0119,  ..., -0.0002,  0.0002,  0.0002]),
 'lab': 'True'}

In [122]:
from speechbrain.lobes.models.ResNet import ResNet
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pretrained_model = ResNet(input_size=80, 
               device=device, 
               channels=[128, 128, 256, 256], 
               block_sizes=[3, 4, 6, 3], 
               strides=[1, 2, 2, 2], 
               lin_neurons=256)

pretrainer = speechbrain.utils.parameter_transfer.Pretrainer(loadables={'model': pretrained_model}, paths={'model': 'speechbrain/spkrec-resnet-voxceleb/embedding_model.ckpt'})
pretrainer.collect_files
pretrainer.load_collected()

pretrained_model.eval()
pretrained_model.to(device)


ResNet(
  (conv1): Conv2d(1, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activation1): ReLU()
  (layer1): Sequential(
    (0): SEBasicBlock(
      (activation): ReLU()
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (se): SEBlock(
        (avg_pool): AdaptiveAvgPool2d(output_size=1)
        (fc): Sequential(
          (0): Linear(in_features=128, out_features=128, bias=Tr

In [ ]:
predicted_labels = []
scores = []
true_labels = []
outputs = []

feature_maker = Fbank(n_mels = 80)
norm = InputNormalization(norm_type = 'sentence', std_norm = False)
cosine_sim = torch.nn.CosineSimilarity(dim=-1, eps=1e-6)
finetuned_model.eval()
interpolated_model.eval()
for batch in iter(dataloader_test):
    batch = batch.to(device)
    signal1, lens1 = batch.signal1
    signal2, lens2 = batch.signal2
    labels = batch.lab

    # Convert labels to tensor and move to GPU
    # labels = torch.tensor(labels, dtype=torch.long).to(device)
    if len(signal1.shape) == 1:
        signal1 = signal1.unsqueeze(0)

    if lens1 is None:
        lens1 = torch.ones(signal1.shape[0], device=device)

    if len(signal2.shape) == 1:
        signal2 = signal2.unsqueeze(0)

    if lens2 is None:
        lens2 = torch.ones(signal2.shape[0], device=device)

    signal1, lens1 = signal1.to(device), lens1.to(device)
    signal1 = signal1.float()

    signal2, lens2 = signal2.to(device), lens2.to(device)
    signal2 = signal2.float()

    features1 = feature_maker(signal1)
    features2 = feature_maker(signal2)

    features1 = norm(features1, lens1)
    features2 = norm(features2, lens2)

    with torch.no_grad():
        embeddings1 = interpolated_model(features1)
        embeddings2 = interpolated_model(features2)

        # embeddings1 = torch.nn.functional.normalize(embeddings1, dim=-1).cpu()
        # embeddings2 = torch.nn.functional.normalize(embeddings2, dim=-1).cpu()

    # Compute cosine similarity and classify
    score = cosine_sim(embeddings1, embeddings2)
    prediction = (score > 0.25).long()
    
    # Threshold can be adjusted
    outputs.append((score, prediction, batch["lab"]))
    # Convert predictions to labels
#     batch_predicted_labels = [True if pred == 1 else False for pred in predictions]
    # predicted_labels.extend(predictions.to('cpu'))
    # true_labels.extend(labels.to('cpu'))
    # scores.extend(similarities.to('cpu'))

#     outputs.append((score, prediction, batch["label"]))

/home/ivbeveren/.local/lib/python3.11/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
/home/ivbeveren/.local/lib/python3.11/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with an OutOfMemoryError: CUDA out of memory. Tried to allocate 6.96 GiB. GPU  (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:924.)
  return F.conv2d(input, weight, bias, self.stride,
/home/ivbeveren/.local/lib/python3.11/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with an OutOfMemoryError: CUDA out of memory. Tried to allocate 13.92 GiB. GPU  (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:924.)
  return F.conv2d(input, weight, bias, self.stride,
/home/iv

In [99]:
scores, X, y = zip(*outputs)

In [100]:

y = sum(list(y), [])
X = [value for sublist in X for value in sublist]
X = [t.item() for t in X]

scores = [value for sublist in scores for value in sublist]
scores = [t.item() for t in scores]

def str_to_bool(s):
    if s == 'True':
         return True
    elif s == 'False':
         return False

y = list(map(str_to_bool, y))

X

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [101]:
def str_to_bool(s):
    if s == 'True':
         return True
    elif s == 'False':
         return False

In [102]:
true_labels

[]

In [103]:
import pandas as pd
import numpy as np
import os
from operator import itemgetter
from scipy.optimize import brentq
from sklearn.metrics import roc_curve
from scipy.interpolate import interp1d

def calculate_eer(y, y_score, pos):
# y denotes groundtruth scores,
# y_score denotes the prediction scores.

    fpr, tpr, thresholds = roc_curve(y, y_score, pos_label=pos)
    eer = brentq(lambda x : 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
    thresh = interp1d(fpr, thresholds)(eer)

    return eer, thresh, fpr, tpr

# Creates a list of false-negative rates, a list of false-positive rates
# and a list of decision thresholds that give those error-rates.
def ComputeErrorRates(scores, labels):

    # Sort the scores from smallest to largest, and also get the corresponding
    # indexes of the sorted scores.  We will treat the sorted scores as the
    # thresholds at which the the error-rates are evaluated.
    sorted_indexes, thresholds = zip(*sorted(
        [(index, threshold) for index, threshold in enumerate(scores)],
        key=itemgetter(1)))
    sorted_labels = []
    labels = [labels[i] for i in sorted_indexes]
    fnrs = []
    fprs = []

    # At the end of this loop, fnrs[i] is the number of errors made by
    # incorrectly rejecting scores less than thresholds[i]. And, fprs[i]
    # is the total number of times that we have correctly accepted scores
    # greater than thresholds[i].
    for i in range(0, len(labels)):
        if i == 0:
            fnrs.append(labels[i])
            fprs.append(1 - labels[i])
        else:
            fnrs.append(fnrs[i-1] + labels[i])
            fprs.append(fprs[i-1] + 1 - labels[i])
    fnrs_norm = sum(labels)
    fprs_norm = len(labels) - fnrs_norm

    # Now divide by the total number of false negative errors to
    # obtain the false positive rates across all thresholds
    fnrs = [x / float(fnrs_norm) for x in fnrs]

    # Divide by the total number of corret positives to get the
    # true positive rate.  Subtract these quantities from 1 to
    # get the false positive rates.
    fprs = [1 - x / float(fprs_norm) for x in fprs]
    return fnrs, fprs, thresholds

# Computes the minimum of the detection cost function.  The comments refer to
# equations in Section 3 of the NIST 2016 Speaker Recognition Evaluation Plan.
def ComputeMinDcf(fnrs, fprs, thresholds, p_target, c_miss, c_fa):
    min_c_det = float("inf")
    min_c_det_threshold = thresholds[0]
    for i in range(0, len(fnrs)):
        # See Equation (2).  it is a weighted sum of false negative
        # and false positive errors.
        c_det = c_miss * fnrs[i] * p_target + c_fa * fprs[i] * (1 - p_target)
        if c_det < min_c_det:
            min_c_det = c_det
            min_c_det_threshold = thresholds[i]
    # See Equations (3) and (4).  Now we normalize the cost.
    c_def = min(c_miss * p_target, c_fa * (1 - p_target))
    min_dcf = min_c_det / c_def
    return min_dcf, min_c_det_threshold

In [104]:
eer, thresh, fpr, tpr = calculate_eer(y, scores, pos=1)
print('EER : %.2f%%'%(eer*100))

EER : 17.00%


In [105]:
c_miss = 1
c_fa = 1
p_target = 0.01

fnrs, fprs, thresholds = ComputeErrorRates(scores,
                                           y)
mindcf, threshold_default = ComputeMinDcf(fnrs, fprs, thresholds, p_target, c_miss, c_fa)

print("minDCF : {0:.4f}, at threshold {1:.4f} (p-target={2}, c-miss={3}, "
    "c-fa={4})".format(mindcf, threshold_default, p_target,c_miss, c_fa))

minDCF : 0.5584, at threshold 0.7521 (p-target=0.01, c-miss=1, c-fa=1)


In [52]:
torch.save(finetuned_model.state_dict(), "best_finetuned_model.pth")

In [21]:
def predict(model):
    predicted_labels = []
    scores = []
    true_labels = []
    outputs = []
    
    feature_maker = Fbank(n_mels = 80)
    norm = InputNormalization(norm_type = 'sentence', std_norm = False)
    cosine_sim = torch.nn.CosineSimilarity(dim=-1, eps=1e-6)
    model.eval()
    for batch in iter(dataloader_test):
        batch = batch.to(device)
        signal1, lens1 = batch.signal1
        signal2, lens2 = batch.signal2
        labels = batch.lab
    
        # Convert labels to tensor and move to GPU
        # labels = torch.tensor(labels, dtype=torch.long).to(device)
        if len(signal1.shape) == 1:
            signal1 = signal1.unsqueeze(0)
    
        if lens1 is None:
            lens1 = torch.ones(signal1.shape[0], device=device)
    
        if len(signal2.shape) == 1:
            signal2 = signal2.unsqueeze(0)
    
        if lens2 is None:
            lens2 = torch.ones(signal2.shape[0], device=device)
    
        signal1, lens1 = signal1.to(device), lens1.to(device)
        signal1 = signal1.float()
    
        signal2, lens2 = signal2.to(device), lens2.to(device)
        signal2 = signal2.float()
    
        features1 = feature_maker(signal1)
        features2 = feature_maker(signal2)
    
        features1 = norm(features1, lens1)
        features2 = norm(features2, lens2)
    
        with torch.no_grad():
            embeddings1 = model(features1)
            embeddings2 = model(features2)
    
            # embeddings1 = torch.nn.functional.normalize(embeddings1, dim=-1).cpu()
            # embeddings2 = torch.nn.functional.normalize(embeddings2, dim=-1).cpu()
    
        # Compute cosine similarity and classify
        score = cosine_sim(embeddings1, embeddings2)
        prediction = (score > 0.25).long()
        
        # Threshold can be adjusted
        outputs.append((score, prediction, batch["lab"]))

    scores, X, y = zip(*outputs)
    y = sum(list(y), [])
    X = [value for sublist in X for value in sublist]
    X = [t.item() for t in X]
    
    scores = [value for sublist in scores for value in sublist]
    scores = [t.item() for t in scores]
    y = list(map(str_to_bool, y))
    c_miss = 1
    c_fa = 1
    p_target = 0.01
    eer, thresh, fpr, tpr = calculate_eer(y, scores, pos=1)
    fnrs, fprs, thresholds = ComputeErrorRates(scores,
                                           y)
    mindcf, threshold_default = ComputeMinDcf(fnrs, fprs, thresholds, p_target, c_miss, c_fa)
    return eer, mindcf


In [22]:
results = []
alphas = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
for alpha in alphas:
    interpolated_model = interpolate_models(pretrained_model, finetuned_model, alpha)
    eer, mindcf = predict(interpolated_model)
    results.append((alpha, eer, mindcf))
    print("Results at alpha = " + str(alpha) + ": " + str(eer) + ", " + str(mindcf))

Results at alpha = 0.1: 0.006681514476614676, 0.0200445434298441
Results at alpha = 0.2: 0.00655021833976097, 0.013363028953229397
Results at alpha = 0.3: 0.004454342984574699, 0.011135857461024499
Results at alpha = 0.4: 0.004454342984409785, 0.013363028953229397
Results at alpha = 0.5: 0.006681514476688931, 0.011135857461024499
Results at alpha = 0.6: 0.006681514476614663, 0.013363028953229397
Results at alpha = 0.7: 0.006681514476859302, 0.015590200445434297
Results at alpha = 0.8: 0.008908685968819875, 0.024498886414253896
Results at alpha = 0.9: 0.01091703056791552, 0.028953229398663696


In [61]:
!pip install seaborn

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 32.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [74]:
results1 = [
    (0.1, 0.006681514476614676, 0.0200445434298441),
    (0.2, 0.00655021833976097, 0.013363028953229397),
    (0.3, 0.004454342984574699, 0.011135857461024499),
    (0.4, 0.004454342984409785, 0.013363028953229397),
    (0.5, 0.006681514476688931, 0.011135857461024499),
    (0.6, 0.006681514476614663, 0.013363028953229397),
    (0.7, 0.006681514476859302, 0.015590200445434297),
    (0.8, 0.008908685968819875, 0.024498886414253896),
    (0.9, 0.01091703056791552, 0.028953229398663696)
]

In [75]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="whitegrid")
results = [(alpha, EER * 10, minDCF) for alpha, EER, minDCF in results]
alpha, EER, minDCF = zip(*results)

# Plotting EER vs alpha
plt.figure(figsize=(10, 5))
sns.lineplot(x=alpha, y=EER, marker='o', color='b')
plt.title('ResNet WSE EER for different values of alpha on UCLA validation')
plt.xlabel('Alpha')
plt.ylabel('EER (%)')
plt.savefig('EER.png')

In [77]:
plt.figure(figsize=(10, 5))
sns.lineplot(x=alpha, y=minDCF, marker='o', color='b')
plt.title('ResNet WSE minDCF for different values of alpha on UCLA validation')
plt.xlabel('Alpha')
plt.ylabel('MinDCF')
plt.savefig('minDCF.png')

In [82]:
results = []
alphas = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
for alpha in alphas:
    interpolated_model = interpolate_models(pretrained_model, finetuned_model, alpha)
    eer, mindcf = predict(interpolated_model)
    results.append((alpha, eer, mindcf))
    print("Results at alpha = " + str(alpha) + ": " + str(eer) + ", " + str(mindcf))

/home/ivbeveren/.local/lib/python3.11/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


Results at alpha = 0.1: 0.053872053870595626, 0.31200897867564487
Results at alpha = 0.2: 0.052749719416385794, 0.25252525252525204
Results at alpha = 0.3: 0.051627384961151476, 0.2289562289562285
Results at alpha = 0.4: 0.0516273849607206, 0.22671156004489285
Results at alpha = 0.5: 0.04826038159371492, 0.22671156004489285
Results at alpha = 0.6: 0.04713804713762575, 0.22671156004489285
Results at alpha = 0.7: 0.043771043772440565, 0.23232323232323182
Results at alpha = 0.8: 0.04489337822665825, 0.2334455667788996
Results at alpha = 0.9: 0.044893378226711585, 0.24915824915824863


In [83]:
results1 = [
    (0.1, 0.006681514476614676, 0.0200445434298441),
    (0.2, 0.00655021833976097, 0.013363028953229397),
    (0.3, 0.004454342984574699, 0.011135857461024499),
    (0.4, 0.004454342984409785, 0.013363028953229397),
    (0.5, 0.006681514476688931, 0.011135857461024499),
    (0.6, 0.006681514476614663, 0.013363028953229397),
    (0.7, 0.006681514476859302, 0.015590200445434297),
    (0.8, 0.008908685968819875, 0.024498886414253896),
    (0.9, 0.01091703056791552, 0.028953229398663696)
]

# New data
results2 = [
    (0.1, 0.053872053870595626, 0.31200897867564487),
    (0.2, 0.052749719416385794, 0.25252525252525204),
    (0.3, 0.051627384961151476, 0.2289562289562285),
    (0.4, 0.0516273849607206, 0.22671156004489285),
    (0.5, 0.04826038159371492, 0.22671156004489285),
    (0.6, 0.04713804713762575, 0.22671156004489285),
    (0.7, 0.043771043772440565, 0.23232323232323182),
    (0.8, 0.04489337822665825, 0.2334455667788996),
    (0.9, 0.044893378226711585, 0.24915824915824863)
]

# Multiply all EER values by 100
results1 = [(alpha, EER * 100, minDCF) for alpha, EER, minDCF in results1]
results2 = [(alpha, EER * 100, minDCF) for alpha, EER, minDCF in results2]

# Unpacking the updated results into separate lists
alpha1, EER1, minDCF1 = zip(*results1)
alpha2, EER2, minDCF2 = zip(*results2)

# Set the style of seaborn
sns.set(style="whitegrid")

# Plotting EER vs alpha
plt.figure(figsize=(10, 5))
sns.lineplot(x=alpha1, y=EER1, marker='o', color='b', label='UCLA')
sns.lineplot(x=alpha2, y=EER2, marker='o', color='g', label='Commonvoice')
plt.title('ResNet WSE EER for different values of alpha on UCLA and Commonvoice')
plt.xlabel('Alpha')
plt.ylabel('EER')
plt.legend()
plt.savefig('EER2.png')

# Plotting minDCF vs alpha
plt.figure(figsize=(10, 5))
sns.lineplot(x=alpha1, y=minDCF1, marker='o', color='b', label='UCLA')
sns.lineplot(x=alpha2, y=minDCF2, marker='o', color='g', label='Commonvoice')
plt.title('ResNet WSE minDCF for different values of alpha on UCLA and Commonvoice')
plt.xlabel('Alpha')
plt.ylabel('minDCF')
plt.legend()
plt.savefig('minDCF2.png')